# Classificação multilabel de textos: múltiplos contextos em NLP

## Objetivos:
* Aprenda o que é a classificação multilabel.
* Conheça a diferença entre a classificação multilabel e classificação multiclass.
* Aplique classificação multilabel para classificar tags das questões do Stackoverflow.
* Aprenda as métricas de avaliações para modelos de classificação multilabel.
* Conheça o Scikit-Multilearn, biblioteca específica para trabalhar com classificação multilabel.

link: https://cursos.alura.com.br/course/classificacao-multilabel-nlp

## Importando as bibliotecas necessárias

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import hamming_loss

from skmultilearn.problem_transform import ClassifierChain, BinaryRelevance
from skmultilearn.adapt import MLkNN

## Carregando os dados

In [2]:
df = pd.read_csv('dataset/stackoverflow_perguntas.csv')
df.head()

,Perguntas,Tags
0,Possuo um projeto Node.js porém preciso criar ...,node.js
1,"Gostaria de fazer testes unitários no Node.js,...",node.js
2,Como inverter a ordem com que o jQuery itera u...,jquery
3,Eu tenho uma página onde pretendo utilizar um ...,html
4,Como exibir os dados retornados do FireStore e...,html angular


In [3]:
print(f'O conjunto de dados é possui {df.shape[0]} elementos e {df.shape[1]} atributos')

O conjunto de dados é possui 5408 elementos e 2 atributos


In [4]:
df.Tags.nunique()

37

In [5]:
df.Tags.str.strip().unique()

array(['node.js', 'jquery', 'html', 'html angular', 'angular',
       'jquery html', 'angular node.js', 'jquery html angular',
       'html jquery', 'jquery angular', 'html node.js', 'jquery node.js',
       'node.js html', 'angular html', 'html angular  node.js',
       'jquery html node.js', 'html angular node.js'], dtype=object)

In [6]:
lista_de_tags = set(np.concatenate([i.split() for i in df.Tags.str.strip().unique()]).flat)

## Criando uma coluna para cada tag do dataframe

In [7]:
def tags_categoricas(tags):
    dic = dict()
    for tag in tags.split():
        dic[tag] = 1
    return dic

dic = df.Tags.apply(tags_categoricas)

colunas_de_tags = pd.DataFrame.from_records(dic).fillna(0).astype('int')
df = pd.concat([df, colunas_de_tags], axis=1).drop('Tags', axis=1)
df.sample(10)

,Perguntas,node.js,jquery,html,angular
1791,Tenho um site e necessito inserir um mapa do a...,0,0,1,0
4268,"Olá, já postei diversas mensagens sobre o assu...",1,0,1,0
4890,Preciso enviar dados de um formulário via ajax...,0,1,1,0
2488,Estou com uma dúvida e está meio difícil encon...,0,0,1,0
1854,Preciso que um GIF fique funcionando enquanto ...,0,1,1,0
1161,Onde está o erro aqui? Gostaria que a imagem a...,0,0,1,0
1513,Tenho uma função chamada por dois elementos HT...,0,1,0,0
674,Como verificar se há algum elemento visível ou...,0,1,0,0
3683,Estou tentando passar o CODE do CODE dentro ...,0,0,0,1
1905,Galera montei um sistema de abas utilizando jQ...,0,1,1,0


## Criando uma coluna com todas as tags que serão os labels do modelo

In [8]:
lista_de_tags = list(lista_de_tags)
lista_zip_tags = list(zip(df[lista_de_tags[0]],
                          df[lista_de_tags[1]],
                          df[lista_de_tags[2]],
                          df[lista_de_tags[3]]))
df['todas_tags'] = lista_zip_tags
df.sample(10)

,Perguntas,node.js,jquery,html,angular,todas_tags
3363,Estou usando o seguinte código: CODE O prob...,0,1,0,0,"(1, 0, 0, 0)"
2973,Estou criando um app de smartphone usando Cord...,0,1,1,0,"(1, 0, 0, 1)"
555,Estou iniciando meus estudos em Angular 7 e em...,0,0,1,1,"(0, 0, 1, 1)"
4204,Tenho uma API e estou a tentar conectar com um...,1,0,0,0,"(0, 1, 0, 0)"
1951,Neste script a baixo ele faz uma alternancia e...,0,1,0,0,"(1, 0, 0, 0)"
4656,O que desejo fazer O cara tem no formulário a...,0,1,0,0,"(1, 0, 0, 0)"
603,Ao atualizar a biblioteca SweetAlert ela parou...,0,1,0,0,"(1, 0, 0, 0)"
3030,CODE,0,1,1,0,"(1, 0, 0, 1)"
2970,Existe algum plugin em jQuery que faz este tip...,0,1,0,0,"(1, 0, 0, 0)"
207,Gostaria de formatar texto na seguinte linha: ...,0,1,0,0,"(1, 0, 0, 0)"


In [9]:
X = df['Perguntas']
y = df.todas_tags

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=123)

y_train_array = np.asarray(list(y_train))
y_test_array = np.asarray(list(y_test))

In [10]:
vetorizador = TfidfVectorizer(max_features=5000, max_df=0.85)

vetorizador.fit(X)
X_train_tfidf = vetorizador.transform(X_train)
X_test_tfidf = vetorizador.transform(X_test)
print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(4326, 5000)
(1082, 5000)


## 1º Modelo - Binary Relevance (OneVsRest)
Nesse modelo é criado um modelo indepedente para cada label

In [27]:
regressao_logistica = LogisticRegression()
classificador_onevsrest = OneVsRestClassifier(regressao_logistica)
classificador_onevsrest.fit(X_train_tfidf, y_train_array)
score_onevsrest = classificador_onevsrest.score(X_test_tfidf, y_test_array)

print(f'O score do modelo OneVsRestClassifier é: {score_onevsrest:0.2%}')

O score do modelo OneVsRestClassifier é: 41.68%


In [28]:
y_pred_onevsrest = classificador_onevsrest.predict(X_test_tfidf)
score_onevsrest_hamming_loss = hamming_loss(y_test_array, y_pred_onevsrest)

print(f'O hamming loss do modelo OneVsRest é: {score_onevsrest_hamming_loss:0.3f}')

O hamming loss do modelo OneVsRest é: 0.188


In [29]:
df.corr()

,node.js,jquery,html,angular
node.js,1.000000,-0.321485,-0.273523,-0.101787
jquery,-0.321485,1.000000,-0.253977,-0.366269
html,-0.273523,-0.253977,1.000000,-0.286706
angular,-0.101787,-0.366269,-0.286706,1.000000


## 2º modelo - Classificação em cadeia
Nesse modelo a correlação entre as tags é levado em consideração para o classificador

In [30]:
regressao_logistica = LogisticRegression()
classificador_cadeia = ClassifierChain(regressao_logistica)
classificador_cadeia.fit(X_train_tfidf, y_train_array)
score_cadeia = classificador_cadeia.score(X_test_tfidf, y_test_array)

print(f'A acuracia do modelo classificador em cadeia é: {score_cadeia:0.2%}')

A acuracia do modelo classificador em cadeia é: 52.31%


In [31]:
y_pred_cadeia = classificador_cadeia.predict(X_test_tfidf)
score_cadeia_hamming_loss = hamming_loss(y_test_array, y_pred_cadeia)

print(f'O hamming loss do modelo de classificador em cadeia é: {score_cadeia_hamming_loss:0.3f}')

O hamming loss do modelo de classificador em cadeia é: 0.201


## 3º Modelo - Binary Relevance
Utilizando o mesmo modelo do sklearn porem da biblioteca skmultilearn

In [32]:
regressao_logistica = LogisticRegression()
classificador_br = BinaryRelevance(regressao_logistica)
classificador_br.fit(X_train_tfidf, y_train_array)
score_br = classificador_br.score(X_test_tfidf, y_test_array)

print(f'A acuracia do modelo de relevância binaria é: {score_br:0.2%}')

A acuracia do modelo de relevância binaria é: 41.68%


In [34]:
y_pred_br = classificador_br.predict(X_test_tfidf)
hamming_loss_br = hamming_loss(y_test_array, y_pred_br)

print(f'O hamming loss do modelo de classificador em cadeia é: {hamming_loss_br:0.3f}')

O hamming loss do modelo de classificador em cadeia é: 0.188


## 4º Modelo - Multilabel K Nearest Neighbours

In [47]:
classificador_mlknn = MLkNN(k=10)
classificador_mlknn.fit(X_train_tfidf, y_train_array)
score_mlknn = classificador_mlknn.score(X_test_tfidf, y_test_array)

y_pred_mlknn = classificador_mlknn.predict(X_test_tfidf)
hamming_loss_mlknn = hamming_loss(y_test_array, y_pred_mlknn)

print(f'A acuracia do modelo mlknn é: {score_mlknn:0.2%}')
print(f'O hamming loss do modelo mlknn é: {hamming_loss_mlknn:0.3f}')

E:\Anaconda3\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


A acuracia do modelo mlknn é: 32.53%
O hamming loss do modelo mlknn é: 0.252


## Resumo

In [69]:
resultado_classificacoes = pd.DataFrame()
resultado_classificacoes['Perguntas'] = X_test.values
resultado_classificacoes['Tags reais'] = y_test.values
resultado_classificacoes['Previsto cadeia'] = list(y_pred_cadeia.toarray().astype('int'))
resultado_classificacoes['Previsto br'] = list(y_pred_br.toarray())
resultado_classificacoes['Previsto mlknn'] = list(y_pred_mlknn.toarray())

resultado_classificacoes.head()

,Perguntas,Tags reais,Previsto cadeia,Previsto br,Previsto mlknn
0,estou com conflito entre o CODE e os CODE ...,"(1, 0, 0, 0)","[1, 0, 0, 0]","[1, 0, 0, 0]","[0, 0, 0, 0]"
1,Estou fazendo um site que eu sou obrigado a us...,"(0, 0, 0, 1)","[0, 0, 0, 1]","[0, 0, 0, 1]","[1, 0, 0, 1]"
2,Recentemente fiz um refactor do meu código par...,"(0, 1, 0, 0)","[0, 1, 0, 0]","[0, 1, 0, 0]","[0, 1, 0, 0]"
3,Eu tenho esse código em CODE que passo valore...,"(1, 0, 0, 1)","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 1]"
4,"Olá, em minha função tem o evento CODE que de...","(1, 0, 0, 1)","[1, 0, 0, 0]","[1, 0, 0, 0]","[1, 0, 0, 1]"
